<a href="https://colab.research.google.com/github/AbimaelLopez/Diplomado/blob/main/Covid19DatosClasificacionComentado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Como primer paso se cargan los datos obtenidos para covid de la página de datos abiertos de gobierno:

https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico

Estos datos contienen la información de contagios en el pais durante el año 2023 y 2024

In [ ]:

#Se importan las bibliotecas a utilizar en nuestro codigo.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

url = "/content/drive/MyDrive/COVID19MEXICO.csv"

covid_data = pd.read_csv(url)


print(covid_data.describe())

Mounted at /content/drive
             ORIGEN        SECTOR    ENTIDAD_UM          SEXO   ENTIDAD_NAC  \
count  1.273091e+06  1.273091e+06  1.273091e+06  1.273091e+06  1.273091e+06   
mean   1.645518e+00  7.115490e+00  1.462528e+01  1.412806e+00  1.542452e+01   
std    4.783563e-01  3.846508e+00  7.831342e+00  4.923388e-01  9.544657e+00   
min    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
25%    1.000000e+00  4.000000e+00  9.000000e+00  1.000000e+00  9.000000e+00   
50%    2.000000e+00  4.000000e+00  1.300000e+01  1.000000e+00  1.400000e+01   
75%    2.000000e+00  1.200000e+01  2.100000e+01  2.000000e+00  2.100000e+01   
max    2.000000e+00  9.900000e+01  3.200000e+01  2.000000e+00  9.900000e+01   

        ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE      INTUBADO      NEUMONIA  \
count  1.273091e+06   1.273091e+06   1.273091e+06  1.273091e+06  1.273091e+06   
mean   1.480403e+01   3.867373e+01   1.102555e+00  8.727177e+01  2.397474e+00   
std    7.793062e+00

Esto nos sirve para identificar las columnas de nuestros datos.

In [ ]:
for col in covid_data.columns:
    print(col)

FECHA_ACTUALIZACION
ID_REGISTRO
ORIGEN
SECTOR
ENTIDAD_UM
SEXO
ENTIDAD_NAC
ENTIDAD_RES
MUNICIPIO_RES
TIPO_PACIENTE
FECHA_INGRESO
FECHA_SINTOMAS
FECHA_DEF
INTUBADO
NEUMONIA
EDAD
NACIONALIDAD
EMBARAZO
HABLA_LENGUA_INDIG
INDIGENA
DIABETES
EPOC
ASMA
INMUSUPR
HIPERTENSION
OTRA_COM
CARDIOVASCULAR
OBESIDAD
RENAL_CRONICA
TABAQUISMO
OTRO_CASO
TOMA_MUESTRA_LAB
RESULTADO_LAB
TOMA_MUESTRA_ANTIGENO
RESULTADO_ANTIGENO
CLASIFICACION_FINAL
MIGRANTE
PAIS_NACIONALIDAD
PAIS_ORIGEN
UCI


Tras evaluar los datos así como la información obtenida para la calidad del aire, se delimitaron los datos a solo aquellos que corresponden a una clasificación de covid positiva, es decir que hubo un contagio de covid, así como a los casos que corresponden a la ciudad de mexico.

Tambien se eliminaron las columnas identificadas como no utiles para la evaluación, como fecha de actualizacion del registro, id de registro, origen, etc.

In [ ]:
#aEsta linea elimina las columnas que no se utilizaran.
covid_data_t = covid_data.drop(columns=['FECHA_ACTUALIZACION', 'ID_REGISTRO','ORIGEN','SEXO','ENTIDAD_NAC','NACIONALIDAD','HABLA_LENGUA_INDIG','INDIGENA','OTRA_COM','OTRO_CASO','TOMA_MUESTRA_LAB','RESULTADO_LAB','TOMA_MUESTRA_ANTIGENO','RESULTADO_ANTIGENO','MIGRANTE','PAIS_NACIONALIDAD','PAIS_ORIGEN','UCI'])
for col in covid_data_t.columns:
    print(col)

#con esta linea delimitamos el dataset a solo los que cumplen con las caracteristicas mencionadas en el texto, contagio positivo en la ciudad de mexico.
covid_data_t = covid_data_t[((covid_data_t.CLASIFICACION_FINAL == 1) | (covid_data_t.CLASIFICACION_FINAL == 2) | (covid_data_t.CLASIFICACION_FINAL==3)) & (covid_data_t.ENTIDAD_RES == 9)]

print(covid_data_t.head)

print(covid_data_t.describe())


SECTOR
ENTIDAD_UM
ENTIDAD_RES
MUNICIPIO_RES
TIPO_PACIENTE
FECHA_INGRESO
FECHA_SINTOMAS
FECHA_DEF
INTUBADO
NEUMONIA
EDAD
EMBARAZO
DIABETES
EPOC
ASMA
INMUSUPR
HIPERTENSION
CARDIOVASCULAR
OBESIDAD
RENAL_CRONICA
TABAQUISMO
CLASIFICACION_FINAL
<bound method NDFrame.head of          SECTOR  ENTIDAD_UM  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  \
8             6           9            9             15              1   
9            12           9            9              8              1   
16           12           9            9             13              1   
34           12           9            9             12              1   
44            6          15            9              5              1   
...         ...         ...          ...            ...            ...   
1271018      12          15            9              7              1   
1272012       9          15            9              4              1   
1272167      12          15            9             12          

***Codigo Obsoleto***

In [ ]:
#este codigo agrupa los contagios por día y municipio en un nuevo dataset, sin embargo ya no se utilizo este dataset.
covid_data_daily = covid_data_t.groupby(by = ['FECHA_SINTOMAS','MUNICIPIO_RES'])['MUNICIPIO_RES'].count().to_frame(name='CANTIDAD').reset_index()

print(covid_data_daily.head)

print(covid_data_daily.describe())

for col in covid_data_daily.columns:
    print(col)

<bound method NDFrame.head of      FECHA_SINTOMAS  MUNICIPIO_RES  CANTIDAD
0        2023-01-01              2        82
1        2023-01-01              3       102
2        2023-01-01              4        13
3        2023-01-01              5       233
4        2023-01-01              6        78
...             ...            ...       ...
5067     2024-02-22             12         1
5068     2024-02-22             14         1
5069     2024-02-22             16         1
5070     2024-02-23             13         1
5071     2024-02-25             12         1

[5072 rows x 3 columns]>
       MUNICIPIO_RES     CANTIDAD
count    5072.000000  5072.000000
mean        9.713328    23.033320
std        14.623354    30.053195
min         2.000000     1.000000
25%         6.000000     4.000000
50%        10.000000    11.000000
75%        13.000000    30.000000
max       999.000000   270.000000
FECHA_SINTOMAS
MUNICIPIO_RES
CANTIDAD


Como parte del tratamiento de datos se realizo una estandarización en el nombre de municipios/alcaldias de nuestro dataset para poder realizar la relación de datos con los obtenidos de calidad del aire.

Las alcaldias sin datos de estacion, ya sea porque no hay una o porque esta fuera de servicio, se identificaron como N/A para posteriormente eliminar estos datos de nuestro conjunto de datos.

In [ ]:
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([2], 'AZCAPOTZALCO')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([3], 'COYOACAN')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([4], 'CUAJIMALPA')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([5], 'GUSTAVOAMADERO')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([6], 'IZTACALCO')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([7], 'IZTAPALAPA')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([8], 'N/A')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([9], 'N/A')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([10], 'ALVAROOBREGON')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([11], 'TLAHUAC')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([12], 'TLALPAN')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([13], 'N/A')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([14], 'BENITOJUAREZ')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([15], 'CUAHUTEMOC')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([16], 'MIGUELHIDALGO')
covid_data_t['MUNICIPIO_RES'] = covid_data_t['MUNICIPIO_RES'].replace([17], 'VENUSTIANOCARRANZA')

Se agregaron las columnas adicionales en las cuales se estará agregando la información de calidad del aire por día, así como una columna para identificar si fallecio o no el paciente.

Y se estandarizaron el formato de fecha para la fecha de sintomas, la cual se utilizará para realizar la relación de nuestros conjunto de datos junto con la Alcaldía.

In [ ]:
print(covid_data_t.head)

print(covid_data_t.describe())

covid_data_t['FECHA_SINTOMAS'] = pd.to_datetime(covid_data_t['FECHA_SINTOMAS'], infer_datetime_format=True)
covid_data_t["CO"]= np.nan
covid_data_t["O3"]= np.nan
covid_data_t["PM25"]= np.nan
covid_data_daily["DEFUNCION"]= np.nan

print(covid_data_t.head())

<bound method NDFrame.head of          SECTOR  ENTIDAD_UM  ENTIDAD_RES   MUNICIPIO_RES  TIPO_PACIENTE  \
8             6           9            9      CUAHUTEMOC              1   
9            12           9            9             N/A              1   
16           12           9            9             N/A              1   
34           12           9            9         TLALPAN              1   
44            6          15            9  GUSTAVOAMADERO              1   
...         ...         ...          ...             ...            ...   
1271018      12          15            9      IZTAPALAPA              1   
1272012       9          15            9      CUAJIMALPA              1   
1272167      12          15            9         TLALPAN              2   
1272326       9          15            9      CUAJIMALPA              1   
1273081       9          15            9   MIGUELHIDALGO              2   

        FECHA_INGRESO FECHA_SINTOMAS   FECHA_DEF  INTUBADO  NEUMONIA 

Aqui se agrega el texto del tratamiento de datos de calidad del aire.

Los datos de calidad del aire obtenidos para cada una de las particulas seleccionadas (CO, O3 y PM 2.5), se relacionaron utilizando la alcaldía y la fecha de la medición, la fecha se estandarizo para evitar errores de formato.

In [ ]:
urlpm25 = "/content/Maximos_PM2-5_2020_-_2024.csv"
urlco = "/content/Alcaldias_CO.csv"
urlo3 ="/content/Maximos_O3_2020_-_2024.csv"

PM25_data = pd.read_csv(urlpm25)
co_data = pd.read_csv(urlco)
o3_data = pd.read_csv(urlo3)
#PM25_data = PM25_data.rename(columns={'IZTAPA': 'IZTAPALAPA'})

print(PM25_data.describe())

#for col in PM25_data.columns:
    #print(col)

#estas lineas estandarizan la columna fecha a formato datetime.
PM25_data['Fecha'] = pd.to_datetime(PM25_data['Fecha'], infer_datetime_format=True)
co_data['Fecha'] = pd.to_datetime(co_data['Fecha'], infer_datetime_format=True)
o3_data['Fecha'] = pd.to_datetime(o3_data['Fecha'], infer_datetime_format=True)

print(PM25_data.head())
print(co_data.head())
print(o3_data.head())


          TLALPAN  BENITOJUAREZ  AZCAPOTZALCO   COYOACAN_1  CUAJIMALPA_1  \
count  269.000000   1389.000000   1069.000000  1366.000000           0.0   
mean    31.650558     35.779698     40.105706    31.554173           NaN   
std     12.275013     14.353920     14.995038    12.763199           NaN   
min      6.000000      6.000000      7.000000     5.000000           NaN   
25%     24.000000     26.000000     30.000000    23.000000           NaN   
50%     31.000000     35.000000     38.000000    30.000000           NaN   
75%     37.000000     43.000000     48.000000    38.000000           NaN   
max     89.000000    144.000000    139.000000   118.000000           NaN   

       GUSTAVOAMADERO  CUAUHTEMOC  IZTACALCO  VENUSTIANOCARRANZA  \
count      633.000000  138.000000        0.0         1005.000000   
mean        41.816746   40.644928        NaN           41.175124   
std         17.584639   13.775468        NaN           15.680423   
min          8.000000   10.000000        Na

<ipython-input-6-f5d4b416a4c2>:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  PM25_data['Fecha'] = pd.to_datetime(PM25_data['Fecha'], infer_datetime_format=True)
<ipython-input-6-f5d4b416a4c2>:16: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  co_data['Fecha'] = pd.to_datetime(co_data['Fecha'], infer_datetime_format=True)
<ipython-input-6-f5d4b416a4c2>:17: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  o3_data['Fecha'] = pd.to_datetime(o3_data['Fecha'], infer_datetime_format=True)


El codigo siguiente se encarga de ingresar los datos de calidad del aire por alcaldía y fecha a nuestro dataset de covid.

Este código se corre por cada particula para asignarlo en la columna correspondiente.

In [ ]:

for ind in PM25_data.index:
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="AZCAPOTZALCO"), ['PM25']] = PM25_data['AZCAPOTZALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="COYOACAN"), ['PM25']] = PM25_data['COYOACAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAJIMALPA"), ['PM25']] = PM25_data['CUAJIMALPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['PM25']] = PM25_data['GUSTAVOAMADERO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTACALCO"), ['PM25']] = PM25_data['IZTACALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTAPALAPA"), ['PM25']] = PM25_data['IZTAPALAPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="ALVAROOBREGON"), ['PM25']] = PM25_data['ALVAROOBREGON'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLAHUAC"), ['PM25']] = PM25_data['TLAHUAC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLALPAN"), ['PM25']] = PM25_data['TLALPAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="BENITOJUAREZ"), ['PM25']] = PM25_data['BENITOJUAREZ'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAHUTEMOC"), ['PM25']] = PM25_data['CUAUHTEMOC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="MIGUELHIDALGO"), ['PM25']] = PM25_data['MIGUELHIDALGO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == PM25_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['PM25']] = PM25_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_t.head())

    SECTOR  ENTIDAD_UM  ENTIDAD_RES   MUNICIPIO_RES  TIPO_PACIENTE  \
8        6           9            9      CUAHUTEMOC              1   
9       12           9            9             N/A              1   
16      12           9            9             N/A              1   
34      12           9            9         TLALPAN              1   
44       6          15            9  GUSTAVOAMADERO              1   

   FECHA_INGRESO FECHA_SINTOMAS   FECHA_DEF  INTUBADO  NEUMONIA  ...  \
8     2023-02-07     2023-02-07  9999-99-99        97         2  ...   
9     2023-02-20     2023-02-18  9999-99-99        97         2  ...   
16    2023-01-04     2023-01-02  9999-99-99        97         2  ...   
34    2023-03-02     2023-03-01  9999-99-99        97         2  ...   
44    2023-01-31     2023-01-30  9999-99-99        97         2  ...   

    INMUSUPR  HIPERTENSION  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  \
8          2             2               2         2              2   
9   

In [ ]:
for ind in o3_data.index:
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="AZCAPOTZALCO"), ['O3']] = o3_data['AZCAPOTZALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="COYOACAN"), ['O3']] = o3_data['COYOACAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAJIMALPA"), ['O3']] = o3_data['CUAJIMALPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['O3']] = o3_data['GUSTAVOAMADERO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTACALCO"), ['O3']] = o3_data['IZTACALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTAPALAPA"), ['O3']] = o3_data['IZTAPALAPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="ALVAROOBREGON"), ['O3']] = o3_data['ALVAROOBREGON'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLAHUAC"), ['O3']] = o3_data['TLAHUAC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLALPAN"), ['O3']] = o3_data['TLALPAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="BENITOJUAREZ"), ['O3']] = o3_data['BENITOJUAREZ'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAHUTEMOC"), ['O3']] = o3_data['CUAUHTEMOC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="MIGUELHIDALGO"), ['O3']] = o3_data['MIGUELHIDALGO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == o3_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['O3']] = o3_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_t.head())

    SECTOR  ENTIDAD_UM  ENTIDAD_RES   MUNICIPIO_RES  TIPO_PACIENTE  \
8        6           9            9      CUAHUTEMOC              1   
9       12           9            9             N/A              1   
16      12           9            9             N/A              1   
34      12           9            9         TLALPAN              1   
44       6          15            9  GUSTAVOAMADERO              1   

   FECHA_INGRESO FECHA_SINTOMAS   FECHA_DEF  INTUBADO  NEUMONIA  ...  \
8     2023-02-07     2023-02-07  9999-99-99        97         2  ...   
9     2023-02-20     2023-02-18  9999-99-99        97         2  ...   
16    2023-01-04     2023-01-02  9999-99-99        97         2  ...   
34    2023-03-02     2023-03-01  9999-99-99        97         2  ...   
44    2023-01-31     2023-01-30  9999-99-99        97         2  ...   

    INMUSUPR  HIPERTENSION  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  \
8          2             2               2         2              2   
9   

In [ ]:
for ind in co_data.index:
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="AZCAPOTZALCO"), ['CO']] = co_data['AZCAPOTZALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="COYOACAN"), ['CO']] = co_data['COYOACAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAJIMALPA"), ['CO']] = co_data['CUAJIMALPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="GUSTAVOAMADERO"), ['CO']] = co_data['GUSTAVOAMADERO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTACALCO"), ['CO']] = co_data['IZTACALCO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="IZTAPALAPA"), ['CO']] = co_data['IZTAPALAPA'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="ALVAROOBREGON"), ['CO']] = co_data['ALVAROOBREGON'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLAHUAC"), ['CO']] = co_data['TLAHUAC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="TLALPAN"), ['CO']] = co_data['TLALPAN'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="BENITOJUAREZ"), ['CO']] = co_data['BENITOJUAREZ'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="CUAHUTEMOC"), ['CO']] = co_data['CUAHUTEMOC'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="MIGUELHIDALGO"), ['CO']] = co_data['MIGUELHIDALGO'][ind]
  covid_data_t.loc[(covid_data_t.FECHA_SINTOMAS == co_data['Fecha'][ind]) & (covid_data_t.MUNICIPIO_RES =="VENUSTIANOCARRANZA"), ['CO']] = co_data['VENUSTIANOCARRANZA'][ind]


print(covid_data_t.head())

    SECTOR  ENTIDAD_UM  ENTIDAD_RES   MUNICIPIO_RES  TIPO_PACIENTE  \
8        6           9            9      CUAHUTEMOC              1   
9       12           9            9             N/A              1   
16      12           9            9             N/A              1   
34      12           9            9         TLALPAN              1   
44       6          15            9  GUSTAVOAMADERO              1   

   FECHA_INGRESO FECHA_SINTOMAS   FECHA_DEF  INTUBADO  NEUMONIA  ...  \
8     2023-02-07     2023-02-07  9999-99-99        97         2  ...   
9     2023-02-20     2023-02-18  9999-99-99        97         2  ...   
16    2023-01-04     2023-01-02  9999-99-99        97         2  ...   
34    2023-03-02     2023-03-01  9999-99-99        97         2  ...   
44    2023-01-31     2023-01-30  9999-99-99        97         2  ...   

    INMUSUPR  HIPERTENSION  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  \
8          2             2               2         2              2   
9   

Tras cargar los datos de calidad del aire, procedemos a realizar la categorización de defuncion utilizando el campo Fecha_def de nuestro dataset.

In [ ]:
covid_data_t["DEFUNCION"]= np.nan

covid_data_t.loc[covid_data_t.FECHA_DEF == '9999-99-99', ['DEFUNCION']] = ['N']
covid_data_t.loc[covid_data_t.FECHA_DEF != '9999-99-99', ['DEFUNCION']] = ['Y']

Eliminamos los datos de las alcaldias sin datos de calidad del aire, identificadas en pasos anteriores como N/A.
Y delimitamos nuevamente nuestros datos a fechas del 2023.

Tras este punto realizamos un backup de los datos en xlsx para realizar un analisis visual en excel y tener los datos respaldados y no tener que correr nuevamente el codigo anterior.

In [ ]:
covid_data_t = covid_data_t.drop(covid_data_t[covid_data_t.MUNICIPIO_RES == 'N/A'].index)
covid_data_t = covid_data_t[covid_data_t.FECHA_SINTOMAS < pd.to_datetime('2024-01-01')]

data = covid_data_t
data.to_excel("/content/covid_daily.xlsx", sheet_name="sheet1", index=False)

In [ ]:
covid_data_t.count()

SECTOR                 105342
ENTIDAD_UM             105342
ENTIDAD_RES            105342
MUNICIPIO_RES          105342
TIPO_PACIENTE          105342
FECHA_INGRESO          105342
FECHA_SINTOMAS         105342
FECHA_DEF              105342
INTUBADO               105342
NEUMONIA               105342
EDAD                   105342
EMBARAZO               105342
DIABETES               105342
EPOC                   105342
ASMA                   105342
INMUSUPR               105342
HIPERTENSION           105342
CARDIOVASCULAR         105342
OBESIDAD               105342
RENAL_CRONICA          105342
TABAQUISMO             105342
CLASIFICACION_FINAL    105342
CO                      78283
O3                      90461
PM25                    68052
DEFUNCION              105342
dtype: int64

Importamos bibliotecas adicionales necesarias para los modelos a correr.

In [7]:
#Importamos las bibliotecas que vamos a utilizar
import math
import random
import pandas as pd
import numpy as np

#Datasets
from sklearn.datasets import load_iris

#Model selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold

#Metrics
from sklearn.metrics import accuracy_score

#Classfier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer

Cargamos nuestro set de datos trabajado y respaldado en XLSX

In [8]:
covid_data_t = pd.read_excel('/content/covid_daily.xlsx')

Tras el analisis visual de nuestros datos, se observó que la distribución entre defunciones y no denfunciones con el dataset original era inmensamente desproporcional, lo cual nos llevo a un overfitting, por lo cual se procedió a realizar algunos pasos adicionales en la limpieza de datos.

Como primer paso se identificaron columnas con valores equivalentes a nulos de acuerdo a la definición de los mismos en el dataset, se realizo una sustitución de estos valores por nulos.



In [9]:
covid_data_t.loc[(covid_data_t.INTUBADO == 97)|(covid_data_t.INTUBADO == 98)|(covid_data_t.INTUBADO == 99) , ['INTUBADO']] = np.nan
covid_data_t.loc[(covid_data_t.NEUMONIA == 97)|(covid_data_t.NEUMONIA == 98)|(covid_data_t.NEUMONIA == 99) , ['NEUMONIA']] = np.nan
covid_data_t.loc[(covid_data_t.EMBARAZO == 97)|(covid_data_t.EMBARAZO == 98)|(covid_data_t.EMBARAZO == 99) , ['EMBARAZO']] = np.nan
covid_data_t.loc[(covid_data_t.DIABETES == 97)|(covid_data_t.DIABETES == 98)|(covid_data_t.DIABETES == 99) , ['DIABETES']] = np.nan
covid_data_t.loc[(covid_data_t.EPOC == 97)|(covid_data_t.EPOC == 98)|(covid_data_t.EPOC == 99) , ['EPOC']] = np.nan
covid_data_t.loc[(covid_data_t.ASMA == 97)|(covid_data_t.ASMA == 98)|(covid_data_t.ASMA == 99) , ['ASMA']] = np.nan
covid_data_t.loc[(covid_data_t.INMUSUPR == 97)|(covid_data_t.INMUSUPR == 98)|(covid_data_t.INMUSUPR == 99) , ['INMUSUPR']] = np.nan
covid_data_t.loc[(covid_data_t.HIPERTENSION == 97)|(covid_data_t.HIPERTENSION == 98)|(covid_data_t.HIPERTENSION == 99) , ['HIPERTENSION']] = np.nan
covid_data_t.loc[(covid_data_t.CARDIOVASCULAR == 97)|(covid_data_t.CARDIOVASCULAR == 98)|(covid_data_t.CARDIOVASCULAR == 99) , ['CARDIOVASCULAR']] = np.nan
covid_data_t.loc[(covid_data_t.OBESIDAD == 97)|(covid_data_t.OBESIDAD == 98)|(covid_data_t.OBESIDAD == 99) , ['OBESIDAD']] = np.nan
covid_data_t.loc[(covid_data_t.RENAL_CRONICA == 97)|(covid_data_t.RENAL_CRONICA == 98)|(covid_data_t.RENAL_CRONICA == 99) , ['RENAL_CRONICA']] = np.nan
covid_data_t.loc[(covid_data_t.TABAQUISMO == 97)|(covid_data_t.TABAQUISMO == 98)|(covid_data_t.TABAQUISMO == 99) , ['TABAQUISMO']] = np.nan


Posteriormente tras hacer la sustitucion de valores, se procedio a eliminar de nuestro dataset todos los datos con algun valor nulo en los campos seleccionados.

In [10]:
covid_data_clean = covid_data_t.dropna(subset=['INTUBADO','NEUMONIA','EMBARAZO','DIABETES','EPOC','ASMA','INMUSUPR','HIPERTENSION','CARDIOVASCULAR','OBESIDAD','RENAL_CRONICA','TABAQUISMO'])

covid_data_clean.describe()

,SECTOR,ENTIDAD_UM,ENTIDAD_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,...,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL,CO,O3,PM25
count,1548.000000,1548.000000,1548.0,1548.0,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,...,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1077.000000,1339.000000,1019.000000
mean,6.050388,9.100129,9.0,2.0,1.968992,1.631783,57.312016,1.960594,1.781008,1.936693,...,1.948966,1.702842,1.928295,1.908269,1.932817,1.970930,2.971576,1.124169,84.349888,40.876349
std,3.086144,1.076721,0.0,0.0,0.173395,0.482477,27.005378,0.194621,0.413697,0.243594,...,0.220137,0.457154,0.258083,0.288739,0.250420,0.168056,0.236803,0.590583,25.133512,23.251295
min,4.000000,3.000000,9.0,2.0,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.030000,4.000000,10.000000
25%,4.000000,9.000000,9.0,2.0,2.000000,1.000000,39.000000,2.000000,2.000000,2.000000,...,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,0.715000,67.000000,29.500000
50%,4.000000,9.000000,9.0,2.0,2.000000,2.000000,64.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,1.020000,85.000000,37.000000
75%,9.000000,9.000000,9.0,2.0,2.000000,2.000000,79.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,1.390000,102.000000,47.250000
max,12.000000,30.000000,9.0,2.0,2.000000,2.000000,103.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,8.560000,169.000000,225.000000


De nuestro dataset procedimos a seleccionar las caracteristicas x y nuestro resultado y para el entrenamiento del modelo.

Las caracteristicas seleccionadas fueron aquellas que se identificaron como condiciones previas al paciente que no necesariamente correspondian a complicaciones relacionadas con el covid, así como datos del lugar donde se trató al paciente junto con las caracteristicas de calidad del aire.

Nuestro resultado es la clasificación de defuncion del paciente, la cual solo puede tener dos valores: Y o N.

In [11]:
Xfull = covid_data_clean[[
    'SECTOR',
    'ENTIDAD_UM',
#    'ENTIDAD_RES',
#    'MUNICIPIO_RES',
#   'TIPO_PACIENTE',
#    'FECHA_INGRESO',
#    'FECHA_SINTOMAS',
#    'INTUBADO',
#    'NEUMONIA',
#    'EDAD',
    'EMBARAZO',
    'DIABETES',
#   'EPOC',
    'ASMA',
#   'INMUSUPR',
    'HIPERTENSION',
    'CARDIOVASCULAR',
    'OBESIDAD',
    'RENAL_CRONICA',
    'TABAQUISMO',
#    'CLASIFICACION_FINAL',
    'CO',
    'O3',
    'PM25']]
yfull = covid_data_clean['DEFUNCION']

Xfull.describe()


#covid_data_t.dtypes

,SECTOR,ENTIDAD_UM,EMBARAZO,DIABETES,ASMA,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CO,O3,PM25
count,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1548.000000,1077.000000,1339.000000,1019.000000
mean,6.050388,9.100129,1.960594,1.781008,1.969638,1.702842,1.928295,1.908269,1.932817,1.970930,1.124169,84.349888,40.876349
std,3.086144,1.076721,0.194621,0.413697,0.171636,0.457154,0.258083,0.288739,0.250420,0.168056,0.590583,25.133512,23.251295
min,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.030000,4.000000,10.000000
25%,4.000000,9.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,0.715000,67.000000,29.500000
50%,4.000000,9.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.020000,85.000000,37.000000
75%,9.000000,9.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.390000,102.000000,47.250000
max,12.000000,30.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,8.560000,169.000000,225.000000


La eliminación de datos nulos hecha en los pasos anteriores nos dio como resultado un dataset más pequeño con un desbalance entre nuestros resultados mucho menos, con lo cual fue posible realizar adecuadamente nuestro modelo.

In [12]:
yfull.describe()

count     1548
unique       2
top          N
freq      1373
Name: DEFUNCION, dtype: object

In [13]:
#subset_size = 1500
#X = Xfull[:subset_size]
#y = yfull[:subset_size]

X = Xfull
y = yfull

print(X.dtypes)
print(y.dtypes)

SECTOR              int64
ENTIDAD_UM          int64
EMBARAZO          float64
DIABETES          float64
ASMA              float64
HIPERTENSION      float64
CARDIOVASCULAR    float64
OBESIDAD          float64
RENAL_CRONICA     float64
TABAQUISMO        float64
CO                float64
O3                float64
PM25              float64
dtype: object
object


El siguiente paso a realizar es separar nuestros datos de prueba de nuestros datos de entrenamiento, en un 60% y 40%, se utilizo la estratificación para mantener la relación entre las clases.

In [14]:
# Split the data into training and testing sets using stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

#X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0, stratify=y_temp)


print(X_train.describe())
print(y_train.describe())
print(X_test.describe())
print(y_test.describe())


           SECTOR  ENTIDAD_UM    EMBARAZO    DIABETES        ASMA  \
count  928.000000  928.000000  928.000000  928.000000  928.000000   
mean     5.998922    9.117457    1.953664    1.776940    1.970905   
std      3.062280    1.253344    0.210326    0.416523    0.168163   
min      4.000000    5.000000    1.000000    1.000000    1.000000   
25%      4.000000    9.000000    2.000000    2.000000    2.000000   
50%      4.000000    9.000000    2.000000    2.000000    2.000000   
75%      8.000000    9.000000    2.000000    2.000000    2.000000   
max     12.000000   30.000000    2.000000    2.000000    2.000000   

       HIPERTENSION  CARDIOVASCULAR    OBESIDAD  RENAL_CRONICA  TABAQUISMO  \
count    928.000000      928.000000  928.000000     928.000000  928.000000   
mean       1.695043        1.931034    1.909483       1.929957    1.969828   
std        0.460637        0.253532    0.287076       0.255357    0.171154   
min        1.000000        1.000000    1.000000       1.000000    

 Aqui definimos como se trataran los valores vacios en las columnas usando imputacion, para las caracteristicas numericas se utilizo la media, para las caracteristicas categoricas se utilizó el valor más frecuente.

In [15]:

# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['float64']).columns
categorical_features = X.select_dtypes(include=['int64','object']).columns

# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create column transformer to apply transformations to numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [25]:
from sklearn import svm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Se definieron 3 modelos de clasificación para este problema:
Support Vector Machine, Vecinos más cercanos y Arbol de decisión.

In [17]:
# each of these models will take a turn as the second Pipeline step
clf1 = svm.SVC()
clf2 = KNeighborsClassifier()
clf3 = DecisionTreeClassifier()

Definimos nuestro pipeline.

In [18]:
# create the Pipeline
pipe = Pipeline([('preprocessor', preprocessor), ('classifier', clf1)])

Definimos los hiperparmetros para cada uno de los modelos a utilizar, los cuales se utilizaran en el pipeline definido anteriormente.

In [19]:

# create the parameter dictionary for clf1
params1 = {}
params1['classifier__kernel']= ['sigmoid', 'rbf', 'poly']
params1['classifier__C'] = [1,5,10]
params1['classifier__gamma']= ['scale', 'auto']
params1['classifier__degree']= [2,3,4]
params1['classifier'] = [clf1]

# create the parameter dictionary for clf2
params2 = {}
params2['classifier__n_neighbors'] =  [3, 5, 7]
params2['classifier__weights'] = ['uniform', 'distance']
params2['classifier'] = [clf2]

# create the parameter dictionary for clf2
params3 = {}
params3['classifier__criterion'] = ['gini', 'entropy']
params3['classifier__max_depth'] = [None, 5, 10, 15]
params3['classifier'] = [clf3]


In [20]:
# create a list of parameter dictionaries
params = [params1, params2, params3]

Utilizando nuestro pipeline definido anteriormente, procedemos a hacer el entrenamiento con nuestros datos, el resultado nos dará cual fue el mejor modelo con los hiperparámetros que hayan dado el mejor resultado.

In [23]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params)
grid.fit(X_train, y_train)
grid.best_params_
print(grid.best_params_)

print("Best score:", grid.best_score_)

results_df = pd.DataFrame(grid.cv_results_)
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]


{'classifier': SVC(C=1, degree=2), 'classifier__C': 1, 'classifier__degree': 2, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Best score: 0.8868526591107235


,params,rank_test_score,mean_test_score,std_test_score
kernel,,,,
"SVC(C=1, degree=2)_1_2_scale_rbf","{'classifier': SVC(C=1, degree=2), 'classifier...",1,0.886853,0.000299
"SVC(C=1, degree=2)_1_2_auto_rbf","{'classifier': SVC(C=1, degree=2), 'classifier...",1,0.886853,0.000299
"SVC(C=1, degree=2)_1_2_auto_poly","{'classifier': SVC(C=1, degree=2), 'classifier...",1,0.886853,0.000299
"SVC(C=1, degree=2)_1_3_scale_rbf","{'classifier': SVC(C=1, degree=2), 'classifier...",1,0.886853,0.000299
"SVC(C=1, degree=2)_1_3_auto_rbf","{'classifier': SVC(C=1, degree=2), 'classifier...",1,0.886853,0.000299
...,...,...,...,...
"SVC(C=1, degree=2)_10_2_scale_sigmoid","{'classifier': SVC(C=1, degree=2), 'classifier...",62,0.830828,0.015329
DecisionTreeClassifier()_gini_15,"{'classifier': DecisionTreeClassifier(), 'clas...",65,0.828678,0.017736
DecisionTreeClassifier()_entropy_15,"{'classifier': DecisionTreeClassifier(), 'clas...",66,0.818936,0.012520


Por último el modelo obtenido en el paso anterior lo utilizamos  con nuestro conjunto de prueba para evaluar su efectividad.

El resultado obtenido fue de 88%.

El modelo seleccionado fue SVC, con los siguientes hiperparametros:

Kernel = rbf
C = 1
degree = 2
gamma = scale


In [28]:
# Access the best model (best estimator) directly
best_model = grid.best_estimator_
best_model_name = best_model.named_steps['classifier'].__class__.__name__
# Now you can use the best_model for further analysis or evaluation
best_predictions = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_predictions)

print(f"Best Model Test Accuracy: {best_accuracy}")

print(f"Best Model: {best_model_name}")
#print("Best Model parameters: {best_model['classifier'].get_params()}")

# Calcular métricas en el conjunto de prueba
print("\nMétricas en el conjunto de prueba:")
print("Accuracy:", accuracy_score(y_test, best_predictions))
print("Precision:", precision_score(y_test, best_predictions,pos_label='Y',zero_division=1))
print("Recall:", recall_score(y_test, best_predictions,pos_label='Y',zero_division=1))
print("F1 Score:", f1_score(y_test, best_predictions,pos_label='Y',zero_division=1))


Best Model Test Accuracy: 0.8870967741935484
Best Model: SVC

Métricas en el conjunto de prueba:
Accuracy: 0.8870967741935484
Precision: 1.0
Recall: 0.0
F1 Score: 0.0
